In [40]:
!pip install -q langchain-openai langchain-huggingface chromadb langchain langchain_core langchain_chroma langchain-community langchain_groq bs4 beautifulsoup4 langchain_community faiss-cpu

In [35]:
%%writefile requirements.txt
langchain
ipykernel
python-dotenv
langchain-community
pypdf
bs4
arxiv
wikipedia
PyMuPDF
langchain-text-splitter
langchain-huggingface

Overwriting requirements.txt


In [36]:
!pip -q install langchain-community

In [37]:
!pip -q install  -r requirements.txt

ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter


In [38]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
llm=ChatGroq(model_name="Llama3-8b-8192")

In [42]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader, PyPDFLoader, WebBaseLoader,ArxivLoader,WikipediaLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_huggingface import HuggingFaceEmbeddings
embeddings_HF=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [29]:
import bs4
loader=WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content","post-title","post-header")))
,
)

docs=loader.load()

In [32]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200)
splits=text_splitter.split_documents(docs)

In [45]:
vectorstore=Chroma.from_documents(documents=splits, embedding=embeddings_HF)

In [46]:
retriever=vectorstore.as_retriever()

In [49]:
#PromptTemplate
system_prompt=(""" your an assistant for QA Tasks
Use the following pieces of indrmation to answer.
If you dont know then do not answer unncessary 
information, always be concise use the context
below
{context}
"""
)
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [51]:
question_answer_chain=create_stuff_documents_chain(
    llm,prompt)
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [52]:
response=rag_chain.invoke({"input":"What is Self Reflection"})

In [54]:
response['answer']

'According to the provided information, Self-Reflection is a vital aspect in autonomous agents that allows them to improve iteratively by refining past action decisions and correcting previous mistakes. It plays a crucial role in real-world tasks where trial and error are inevitable.'

In [57]:
## adding chat history
rag_chain.invoke({"input":"How do we achieve it?"})['answer']

'According to the information provided, Tree of Thoughts (Yao et al. 2023) can be used to achieve task decomposition by:\n\n1. Decomposing the problem into multiple thought steps and generating multiple thoughts per step, creating a tree structure.\n2. Using a search process such as BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\n\nAdditionally, task decomposition can be done using:\n\n1. LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?"\n2. Task-specific instructions; e.g. "Write a story outline." for writing a novel.\n3. Human inputs.'

In [86]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [88]:
history_aware_retriever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)


In [89]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [84]:
#history_aware_retriever

In [91]:
question_answer_chain=create_stuff_documents_chain(llm,qa_prompt)
rag_chain=create_retrieval_chain(history_aware_retriever,question_answer_chain)

In [96]:
from langchain_core.messages import AIMessage, HumanMessage
chat_history=[]
question="What is Self-reflection?"
response1=rag_chain.invoke({"input":question,"chat_history":chat_history})

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1['answer'])
        
    ]
)

question2="what are the common ways to achieve this?"
response2=rag_chain.invoke({"input":question2,"chat_history":chat_history})
print(response2['answer'])
                           
                           

{
    "thoughts": {
        "text": "Common ways to achieve self-reflection include showing two-shot examples to LLM and each example is a pair of (failed trajectory, ideal reflection for guiding future changes in the plan). Additionally, relying on an external classical planner to do long-horizon planning and utilizing the Planning Domain Definition Language (PDDL) as an intermediate interface to describe the planning problem.",
        "reasoning": "Common approaches to self-reflection",
        "plan": "- Show two-shot examples to LLM\n- Use Planning Domain Definition Language (PDDL) for long-horizon planning",
        "criticism": "More research is needed to develop more effective self-reflection mechanisms",
        "speak": "To achieve self-reflection, we can show two-shot examples to LLM or use PDDL for long-horizon planning."
    },
    "command": {
        "name": "none",
        "args": {}
    }
}


In [98]:
chat_history

[HumanMessage(content='What is Self-reflection?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='According to the provided information, Self-reflection is a vital aspect that allows autonomous agents to improve iteratively by refining past action decisions and correcting previous mistakes.', additional_kwargs={}, response_metadata={})]

In [ ]:
#Basechathistory runnables


In [100]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [102]:
store={}
def get_session_history(session_id) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

conversational_rag_chain=RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [103]:
conversational_rag_chain.invoke(
    {"input":"What is task decomposition"},
    config={
        "configurable":{"session_id":"abc123"}
    })['answer']

'Task decomposition is a process in which a complex task is broken down into smaller, more manageable steps or subtasks. This allows an agent, such as a computer program or a human, to plan and execute the task more effectively.'

In [104]:
conversational_rag_chain.invoke(
    {"input":"what are the common ways to achieve this?"},
    config={
        "configurable":{"session_id":"abc123"}
    })['answer']

'According to the provided information, there are three common ways to achieve task decomposition:\n\n1. Using LLM (Large Language Model) with simple prompting, such as "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?"\n2. Using task-specific instructions, such as "Write a story outline." for writing a novel\n3. With human inputs'